In [23]:
import pandas as pd

df = pd.read_csv('../../static_data/processed/traffic_weather_intersections.csv')

/var/folders/lx/pn018tmx0pq2y3z58prkblww0000gn/T/ipykernel_10503/1751821100.py:3: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../../static_data/processed/traffic_weather_intersections.csv')


In [17]:
!pip install --upgrade xgboost
!pip install --upgrade scikit-learn
!pip install --upgrade numpy


[notice] A new release of pip is available: 23.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip


### Negative Sampling

In [ ]:
import pandas as pd
import numpy as np

# Create datetime column
df['datetime'] = pd.to_datetime(df['crash_date'].astype(str) + ' ' + df['crash_time'].astype(str), errors='coerce')

# Get all unique intersections and datetimes
all_intersections = df['nearest_intersection_id'].unique()
date_range = pd.date_range(df['crash_date'].min(), df['crash_date'].max(), freq='H')
all_combos = pd.MultiIndex.from_product([all_intersections, date_range], names=['nearest_intersection_id', 'datetime'])
all_combos_df = all_combos.to_frame(index=False)

# Mark where a crash occurred
df['crash_occurred'] = 1
crash_events = df[['nearest_intersection_id', 'datetime', 'crash_occurred']].drop_duplicates()

# Merge to get all possible (intersection, datetime) and mark positives
full_df = all_combos_df.merge(crash_events, on=['nearest_intersection_id', 'datetime'], how='left')
full_df['crash_occurred'] = full_df['crash_occurred'].fillna(0).astype(int)

# Sample negatives: for each intersection, sample N negatives per positive
negatives = full_df[full_df['crash_occurred'] == 0].groupby('nearest_intersection_id').apply(
    lambda x: x.sample(n=min(len(x), 2 * (full_df[(full_df['nearest_intersection_id'] == x.name) & (full_df['crash_occurred'] == 1)].shape[0])), random_state=42)
).reset_index(drop=True)

positives = full_df[full_df['crash_occurred'] == 1]
final_df = pd.concat([positives, negatives], ignore_index=True)

/var/folders/lx/pn018tmx0pq2y3z58prkblww0000gn/T/ipykernel_10503/1868880825.py:9: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  date_range = pd.date_range(df['crash_date'].min(), df['crash_date'].max(), freq='H')


In [20]:

import sklearn
from sklearn.model_selection import train_test_split
import numpy as np

X = df[['nearest_intersection_id', 'hour', 'day_of_week', 'tavg', 'prcp', 'wspd']].copy()
X['nearest_intersection_id'] = X['nearest_intersection_id'].astype('category')
y = np.ones(len(df))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
import xgboost as xgb
import numpy as np

# Create DMatrix for train and test
dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
dtest = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)

# XGBoost parameters for ranking
params = {
    'objective': 'rank:pairwise',
    'eval_metric': 'ndcg',
    'eta': 0.1,
    'max_depth': 6,
    'tree_method': 'hist'
}

# Train the model and evaluate on test set
model = xgb.train(
    params,
    dtrain,
    num_boost_round=100,
    evals=[(dtest, 'test')]
)

[0]	test-ndcg:1.00000
[1]	test-ndcg:1.00000
[2]	test-ndcg:1.00000
[3]	test-ndcg:1.00000
[4]	test-ndcg:1.00000
[5]	test-ndcg:1.00000
[6]	test-ndcg:1.00000
[7]	test-ndcg:1.00000
[8]	test-ndcg:1.00000
[9]	test-ndcg:1.00000
[10]	test-ndcg:1.00000
[11]	test-ndcg:1.00000
[12]	test-ndcg:1.00000
[13]	test-ndcg:1.00000
[14]	test-ndcg:1.00000
[15]	test-ndcg:1.00000
[16]	test-ndcg:1.00000
[17]	test-ndcg:1.00000
[18]	test-ndcg:1.00000
[19]	test-ndcg:1.00000
[20]	test-ndcg:1.00000
[21]	test-ndcg:1.00000
[22]	test-ndcg:1.00000
[23]	test-ndcg:1.00000
[24]	test-ndcg:1.00000
[25]	test-ndcg:1.00000
[26]	test-ndcg:1.00000
[27]	test-ndcg:1.00000
[28]	test-ndcg:1.00000
[29]	test-ndcg:1.00000
[30]	test-ndcg:1.00000
[31]	test-ndcg:1.00000
[32]	test-ndcg:1.00000
[33]	test-ndcg:1.00000
[34]	test-ndcg:1.00000
[35]	test-ndcg:1.00000
[36]	test-ndcg:1.00000
[37]	test-ndcg:1.00000
[38]	test-ndcg:1.00000
[39]	test-ndcg:1.00000
[40]	test-ndcg:1.00000
[41]	test-ndcg:1.00000
[42]	test-ndcg:1.00000
[43]	test-ndcg:1.0000

# Inference: get risk scores for the test set
risk_scores = model.predict(dtest)
X_test = X_test.copy()
X_test['risk_score'] = risk_scores

# Example: Show top 10 riskiest intersections in the test set
top_risk = X_test.sort_values('risk_score', ascending=False)
print(top_risk[['nearest_intersection_id', 'hour', 'day_of_week', 'risk_score']].head(10))